In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import os

In [ ]:

def ler_arquivo_para_dataframe(caminho_arquivo):
    extensao = os.path.splitext(caminho_arquivo)[1].lower()
    if extensao == '.csv':
        df = pd.read_csv(caminho_arquivo)
    elif extensao in ['.xls', '.xlsx']:
        df = pd.read_excel(caminho_arquivo)
    elif extensao == '.json':
        df = pd.read_json(caminho_arquivo)
    else:
        raise ValueError(f"Formato de arquivo não suportado: {extensao}")
    return df

In [ ]:
data = pd.read_excel('produtos.xlsx')

,product_category_name,price,product_id,product_name
0,perfumaria,10.91,1,Hidratante Corporal Nivea Milk
1,artes,248.00,2,Kit Tinta a Óleo Acrilex com 12 Cores
2,esporte_lazer,79.80,3,Bicicleta Aro 29 Caloi Explorer
3,bebes,112.30,4,Carrinho de Bebê Cosco Moove
4,utilidades_domesticas,37.90,5,Liquidificador Oster Clássico 3 Velocidades
...,...,...,...,...
112645,informatica_acessorios,29.90,112646,Mouse sem Fio Logitech M280
112646,informatica_acessorios,29.90,112647,Mouse sem Fio Logitech M280
112647,informatica_acessorios,29.90,112648,Mouse sem Fio Logitech M280
112648,informatica_acessorios,29.90,112649,Teclado Mecânico Gamer Redragon Kumara


In [87]:
data = data.rename(columns={
    'product_category_name': 'Tipo',
    'price': 'Preço',
    'product_name': 'Nome',
    'product_id': 'ID'
})

data

,Tipo,Preço,ID,Nome
0,perfumaria,10.91,1,Hidratante Corporal Nivea Milk
1,artes,248.00,2,Kit Tinta a Óleo Acrilex com 12 Cores
2,esporte_lazer,79.80,3,Bicicleta Aro 29 Caloi Explorer
3,bebes,112.30,4,Carrinho de Bebê Cosco Moove
4,utilidades_domesticas,37.90,5,Liquidificador Oster Clássico 3 Velocidades
...,...,...,...,...
112645,informatica_acessorios,29.90,112646,Mouse sem Fio Logitech M280
112646,informatica_acessorios,29.90,112647,Mouse sem Fio Logitech M280
112647,informatica_acessorios,29.90,112648,Mouse sem Fio Logitech M280
112648,informatica_acessorios,29.90,112649,Teclado Mecânico Gamer Redragon Kumara


In [88]:
# Verificação e conversão dos tipos das colunas

# Verifica e converte 'ID' para inteiro
if not pd.api.types.is_integer_dtype(data['ID']):
    data['ID'] = pd.to_numeric(data['ID'], errors='coerce').astype('Int64')

# Verifica e converte 'Preço' para float
if not pd.api.types.is_float_dtype(data['Preço']):
    data['Preço'] = pd.to_numeric(data['Preço'], errors='coerce')

# Verifica e converte 'Tipo' para string e limita a 100 caracteres
if data['Tipo'].dtype != 'O':
    data['Tipo'] = data['Tipo'].astype(str)
data['Tipo'] = data['Tipo'].str[:100]

# Verifica e converte 'Nome' para string e limita a 100 caracteres
if data['Nome'].dtype != 'O':
    data['Nome'] = data['Nome'].astype(str)
data['Nome'] = data['Nome'].str[:100]

# Exibe os tipos finais das colunas
data.dtypes

Tipo      object
Preço    float64
ID         int64
Nome      object
dtype: object

In [ ]:
from datetime import datetime

# Supondo que db e Products já estejam definidos e configurados no contexto Flask

# Função para salvar o DataFrame 'data' no banco de dados usando o model Products
def salvar_produtos_no_banco(df):
    produtos = []
    for _, row in df.iterrows():
        if pd.isna(row['Nome']) or pd.isna(row['Preço']):
            continue  # pula registros incompletos
        produto = Products(
            product_name=row['Nome'][:100],
            product_type=row['Tipo'][:100] if not pd.isna(row['Tipo']) else None,
            price=float(row['Preço']),
            created_at=datetime.utcnow()
        )
        produtos.append(produto)
    db.session.bulk_save_objects(produtos)
    db.session.commit()

# Exemplo de uso:
# salvar_produtos_no_banco(data)